In [1]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

/notebooks/pipenv/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.43MB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 121kB/s]
Downloading: 100%|██████████| 10.1k/10.1k [00:00<00:00, 6.50MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 153kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 5.32MB/s]
Downloading: 100%|██████████| 349/349 [00:00<00:00, 470kB/s]
Downloading: 100%|██████████| 438M/438M [00:18<00:00, 23.4MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 38.6kB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 302kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 9.18MB/s]
Downloading: 100%|██████████| 363/363 [00:00<00:00, 435kB/s]
Downloading: 100%|██████████| 13.1k/13.1k [00:00<00:00, 2.10MB/s]
Downloading: 10

In [2]:
import numpy as np

In [3]:
import glob
import csv
kg = []
relations = ['oWant', 'AtLocation', 'oReact', 'xWant', 'ObjectUse', 'HinderedBy', 'oEffect', 'xIntent', 'isFilledBy', 'xAttr', 
'HasSubEvent', 'isAfter', 'HasProperty', 'xReact', 'Desires', 'xEffect', 'xNeed', 'Causes', 'isBefore', 'CapableOf', 'xReason', 
'NotDesires', 'MadeUpOf']
print(len(relations))

print(relations)
files = glob.glob('atomic2020_data/*.tsv')
for file in files:
    with open(file) as in_file:
        tsv_file = csv.reader(in_file, delimiter="\t")
        for i, line in enumerate(tsv_file):
            sentence = ""
            #kg['node'] = line
            head = line[0]
            relation = line[1]
            tail = line[2]
            #print(tail)
            if relation == 'isFilledBy':
                sentence = head.replace('___', tail)
                sentence = sentence.replace('___', '')
            if relation == 'AtLocation':
                sentence = head + ' located at ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'oWant' and tail != 'none':
                sentence = head + ' he want ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xWant' and tail != 'none':
                sentence = head + ' he want ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'oReact' and tail != 'none':
                sentence = head + ' and then ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'ObjectUse' and tail != 'none':
                sentence = head + ' used for ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'HinderedBy' and tail != 'none':
                sentence = head + ', this would not happen if ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'oEffect' and tail != 'none':
                sentence = head + ' so he ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xIntent' and tail != 'none':
                sentence = head + ' in order ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xAttr' and tail != 'none':
                sentence = head + ' so he is ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'HasSubEvent' and tail != 'none':
                sentence = head + ' and ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'isAfter' and tail != 'none':
                sentence = head + ', before ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'HasProperty' and tail != 'none':
                sentence = head + ' is ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xReact' and tail != 'none':
                sentence = head + ' and then ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'Desires' and tail != 'none':
                sentence = head + ' desires ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xEffect' and tail != 'none':
                sentence = head + ' so he ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'xNeed' and tail != 'none':
                sentence = head + ', he need ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'Causes' and tail != 'none':
                sentence = head + ' causes ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'isBefore' and tail != 'none':
                sentence = head + ' and then ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'CapableOf' and tail != 'none':
                sentence = head + ' is capable of ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'NotDesires' and tail != 'none':
                sentence = head + ' does not desire ' + tail
                sentence = sentence.replace('___', '')
            if relation == 'MadeUpOf' and tail != 'none':
                sentence = head + ' is made up from ' + tail
                sentence = sentence.replace('___', '')
            kg.append({'entity': line, 'sentence': sentence})

23
['oWant', 'AtLocation', 'oReact', 'xWant', 'ObjectUse', 'HinderedBy', 'oEffect', 'xIntent', 'isFilledBy', 'xAttr', 'HasSubEvent', 'isAfter', 'HasProperty', 'xReact', 'Desires', 'xEffect', 'xNeed', 'Causes', 'isBefore', 'CapableOf', 'xReason', 'NotDesires', 'MadeUpOf']


In [4]:
from nebula3_database.movie_db import MOVIE_DB
mdb = MOVIE_DB()
mdb.change_db("nebula_visualcomet")

In [5]:
def get_vc(mdb):
        metadata = []
        query = 'FOR doc IN vcomet_kg LIMIT 100 RETURN doc'
        cursor = mdb.db.aql.execute(query)
        for data in cursor:
            #print(data)
            metadata.append(data)
        return(metadata)

In [13]:
events = [] 
vc = get_vc(mdb)
for v in vc:
    events.append((v['event'], v['place']))
print(events)

[('2 is carrying a note book and is walking toward a door', 'inside'), ('3 is wearing a bellhop uniform and smiling', 'inside'), ('1 and 2 eat Chinese takeout on the floor while looking through paperwork', 'in a bedroom'), ('2 holds an egg roll and looks disappointed as she reads a paper in front of her', 'in a bedroom'), ('3 is soaking wet and climbs out of an open manhole', 'at an outdoor cafe'), ('several guests at the cafe watch with some surprise as 3 crawls out of the manhole', 'at an outdoor cafe'), ('20 strolls casually down the street with 23 carrying a parasol', 'at an outdoor cafe'), ('1 opens the door for 3', 'in a courthouse'), ('2 intervenes between 1 and 3', 'in a courthouse'), ('3 was at the door with 1', 'in a courthouse'), ('1 stands next to the car and leans in to talk to 2 and 3', 'in a car'), ('2 stares at 1 while 1 looks into the car', 'in a car'), ('3 turns her body towards 1 as she sits in the car', 'in a car'), ('2 is talking to someone', 'on a sinking ship'), 

In [15]:

# sent1 = model.encode(['PersonX takes ___ to the airport', 'go up to the security office', 'go into the embassy',
#                       'enter into the airport', 'go into the hangar', 'enter the terminal' , 
#                       'in a gatehouse of an airport','outside of a terminal baggage claim', 'at an airport entrance', 'on the sstreet', 
#                       'inside an airport'])
#print(len(sent1[0]))
#sent1_mean = np.mean(sent1, axis=0)
#print(len(sent1_mean))
#input()
for event in events:
    print("Event: ", event[0])
    print("Place: ", event[1])
    sent1 = model.encode(event[0])
    place = model.encode(event[0] + event[1])
    for i in kg:
        #if i['sentence'] != '':
        #if i['sentence'] != '' and (i['entity'][1] == 'isAfter' or i['entity'][1] == 'isBefore'):
        if i['sentence'] != '' and i['entity'][1] == 'isFilledBy':
            sent2 = model.encode(i['sentence'])
            #print(len(sent2))
            #print (sent1, " " , sent2)
            sim = util.dot_score(sent1, sent2)
            if abs(sim.tolist()[0][0]) > 0.4:
            #input()
                print(i['entity'])
                print("Event Similarity:", util.dot_score(sent1, sent2))
        if i['sentence'] != '' and i['entity'][1] == 'AtLocation':
            sent2 = model.encode(i['sentence'])
            #print(len(sent2))
            #print (sent1, " " , sent2)
            sim = util.dot_score(place, sent2)
            if abs(sim.tolist()[0][0]) > 0.4:
            #input()
                print(i['entity'])
                print("Place Similarity:", util.dot_score(place, sent2))

Event:  2 is carrying a note book and is walking toward a door
Place:  inside
['book', 'AtLocation', 'backpack']
Place Similarity: tensor([[0.4129]])
['book', 'AtLocation', 'bedroom']
Place Similarity: tensor([[0.4436]])
['book', 'AtLocation', 'book bag']
Place Similarity: tensor([[0.4480]])
['book', 'AtLocation', 'bookcase']
Place Similarity: tensor([[0.4248]])
['book', 'AtLocation', 'bookshelf']
Place Similarity: tensor([[0.4188]])
['book', 'AtLocation', 'bookshelves']
Place Similarity: tensor([[0.4207]])
['book', 'AtLocation', 'briefcase']
Place Similarity: tensor([[0.4284]])
['book', 'AtLocation', 'classroom']
Place Similarity: tensor([[0.4467]])
['book', 'AtLocation', 'closet']
Place Similarity: tensor([[0.4608]])
['book', 'AtLocation', 'desk']
Place Similarity: tensor([[0.4037]])
['book', 'AtLocation', 'drawer']
Place Similarity: tensor([[0.4726]])
['book', 'AtLocation', 'floor']
Place Similarity: tensor([[0.4633]])
['book', 'AtLocation', 'hands of student']
Place Similarity: ten